# Alex Johny
**CMSC389A - Final Project**

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from keras import optimizers
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

import io

# Importing The Data

**In this section I am importing the data from the csv downloaded off of Kaggle**

In [80]:
columns = [
    'agency_code',
    'applicant_ethnicity',
    'applicant_income_000s',
    'applicant_race_1',
    'applicant_sex',
    'co_applicant_ethnicity',
    'co_applicant_race_1',
    'co_applicant_sex',
    'county_code',
    'lien_status',
    'loan_purpose',
    'loan_type',
    'owner_occupancy',
    'preapproval',
    'property_type',
    'purchaser_type',
    'sequence_number',
    'state_code',
    'hud_median_family_income',
    'loan_amount_000s',
    'number_of_1_to_4_family_units',
    'number_of_owner_occupied_units',
    'minority_population',  
    'population',
    'tract_to_msamd_income']


TRAIN_CSV = 'ny_hmda_2015_Cleaned.csv'
df = pd.read_csv(TRAIN_CSV)
    
df.head(10)

,agency_code,applicant_ethnicity,applicant_income_000s,applicant_race_1,applicant_sex,co_applicant_ethnicity,co_applicant_race_1,co_applicant_sex,county_code,lien_status,...,sequence_number,state_code,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,tract_to_msamd_income,action_taken
0,9,2,97.0,5,2,5,8,5,59.0,1,...,164811,36,109000.0,187,363.0,1817.0,21.139999,5870.0,109.459999,1
1,9,2,200.0,5,1,5,8,5,61.0,1,...,26589,36,71300.0,460,53.0,256.0,45.959999,3512.0,160.600006,1
2,7,2,NaN,3,1,5,8,5,71.0,1,...,141383,36,71300.0,296,2745.0,2586.0,38.990002,8357.0,134.820007,1
3,9,2,202.0,5,1,2,5,2,59.0,1,...,154957,36,109000.0,770,1879.0,2147.0,7.350000,6642.0,165.830002,1
4,9,2,255.0,5,1,5,8,5,59.0,1,...,106528,36,109000.0,648,835.0,676.0,30.059999,2339.0,133.300003,1
5,5,2,100.0,5,1,2,5,2,91.0,2,...,1986,36,82700.0,40,3488.0,2002.0,6.130000,8977.0,110.220001,1
6,9,2,25.0,2,1,5,8,5,29.0,1,...,8633,36,69000.0,57,1891.0,804.0,45.419998,4881.0,48.299999,1
7,9,2,13.0,3,2,5,8,5,29.0,1,...,106963,36,69000.0,38,1686.0,519.0,97.900002,2757.0,42.660000,1
8,9,3,230.0,6,3,5,8,5,93.0,1,...,155085,36,82700.0,159,2414.0,2077.0,14.770000,7442.0,138.440002,1
9,9,3,114.0,6,3,5,8,5,59.0,1,...,106529,36,109000.0,302,363.0,1817.0,21.139999,5870.0,109.459999,1


## Building the model

The best approach for this dataset for determining if an applicant would receieve VISA or not would be a deep neural network

In [87]:
model = Sequential()
model.add(Dense(24, input_dim=24, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_66 (Dense)             (None, 12)                300       
_________________________________________________________________
dense_67 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_68 (Dense)             (None, 1)                 7         
Total params: 985
Trainable params: 985
Non-trainable params: 0
_________________________________________________________________


## Training the Model

In [99]:
frac = 0.80
kSEED = 7
np.random.seed(kSEED)

data = []
for row in df.itertuples():
    label = row[26]
    features = row[1:25]
    tup = (features, label)
    data.append(tup)

data = shuffle(data, random_state=kSEED)
split_index = int(frac * len(data))
train = data[:split_index]
test = data[split_index:]

X_train = np.array([x[0] for x in train]) #features
y_train = np.array([x[1] for x in train]) #label
X_test = np.array([x[0] for x in train])  #features
y_test = np.array([x[1] for x in train])  #label

## Testing the Model

In [100]:
sgd = optimizers.SGD(lr=0.005, clipnorm=1.)
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=10)
scores = model.evaluate(X_test, y_test, verbose=False)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20
 2478/52399 [>.............................] - ETA: 1:24 - loss: nan - acc: 0.0000e+00

KeyboardInterrupt: 